In [1]:
import numpy as np
import os
import shutil

In [5]:
class cfg:
    dir_raw = "/kaggle/working/dataset/stack_raw"
    dir_viz = "/kaggle/working/dataset/stack_viz"
    dir_preds = "/kaggle/working/notebook/train/train01/preds"
    train_ver = "02"
    size_xy = 256
    offset_xy = 128
    size_z = 1
    offset_z = 1
    dir_pseudo = "/kaggle/working/dataset/stack_train02"
    dir_dataset = f"/kaggle/working/dataset/train{train_ver}_xy_{size_xy}_{offset_xy}_z_{size_z}_{offset_z}/"

In [3]:
def clip_based_on_percentile(npy, percentile=0.05):
    stack_len = npy.shape[0]

    upper = round(stack_len * 0.3)
    lower = round(stack_len * 0.7)

    p_low = int(np.percentile(npy[upper:lower], percentile))  # 上下端に近い部分はpercentile計算対象から除外
    p_high = int(np.percentile(npy[upper:lower], 100 - percentile))

    npy = np.clip(npy, p_low, p_high).astype("float32")
    scale = float(p_high - p_low)
    npy = (npy - p_low) / scale
    return npy

In [7]:
kidney = "kidney_1_dense"

images_path = f"{cfg.dir_raw}/{kidney}_images.npy"
images = np.load(images_path)
images = clip_based_on_percentile(images)
images_save_path = f"{cfg.dir_pseudo}/{kidney}_images.npy"
np.save(images_save_path, images)

label_path = f"{cfg.dir_viz}/{kidney}_labels.npy"
shutil.copy(label_path, cfg.dir_pseudo)

'/kaggle/working/dataset/stack_train02\\kidney_1_dense_labels.npy'

In [6]:
kidneys = ["kidney_2_sparse", "kidney_3_sparse", "kidney_9_pseudo"]

for kidney in kidneys:
    print(kidney)
    kidney_p = kidney.replace("sparse", "pseudo")

    images_path = f"{cfg.dir_raw}/{kidney}_images.npy"
    images = np.load(images_path)
    images = clip_based_on_percentile(images)
    images_save_path = f"{cfg.dir_pseudo}/{kidney_p}_images.npy"
    np.save(images_save_path, images)

    if kidney == "kidney_2_sparse":
        preds_path = f"{cfg.dir_preds}/{kidney}_best_preds_th.npy"
        pred = np.load(preds_path)

        label_path = f"{cfg.dir_viz}/{kidney}_labels.npy"
        label = np.load(label_path)

    if kidney == "kidney_3_sparse":
        preds_path = f"{cfg.dir_preds}/{kidney}_best_preds_th.npy"
        pred = np.zeros_like(images)
        pred[:496, :, :] = np.load(preds_path)

        dense_label_path = f"{cfg.dir_viz}/kidney_3_dense_labels.npy"
        sparse_label_path = f"{cfg.dir_viz}/kidney_3_sparse_labels.npy"
        dense_label = np.load(dense_label_path)
        sparse_label = np.load(sparse_label_path)
        label = np.logical_or(sparse_label, dense_label)

        del dense_label, sparse_label

    if kidney == "kidney_9_pseudo":
        preds_path = f"{cfg.dir_preds}/{kidney}_best_preds_th.npy"
        pred = np.load(preds_path)
        label = np.zeros_like(images)

    pseudo_label = np.logical_or(label, pred)
    label_save_path = f"{cfg.dir_pseudo}/{kidney_p}_labels.npy"
    np.save(label_save_path, pseudo_label)

    # break

kidney_2_sparse
kidney_3_sparse
kidney_9_pseudo
